In [36]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer
from datasets import Dataset
from transformers import TrainingArguments , Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model

In [2]:
df = pd.read_csv('/content/archive.zip')
df

,recipe_title,url,record_health,vote_count,rating,description,cuisine,course,diet,prep_time,cook_time,ingredients,instructions,author,tags,category
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,https://www.archanaskitchen.com/roasted-pepper...,good,434,4.958525,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15 M,15 M,Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...,To begin making the Roasted Peppers And Mushro...,Divya Shivaraman,Party Food Recipes|Tea Party Recipes|Mushroom ...,Pizza Recipes
1,Thakkali Gotsu Recipe | Thakkali Curry | Spicy...,https://www.archanaskitchen.com/tomato-gotsu-r...,good,3423,4.932223,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,10 M,20 M,Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...,To begin making Tomato Gotsu Recipe/ Thakkali ...,Archana Doshi,Vegetarian Recipes|Tomato Recipes|South Indian...,Indian Curry Recipes
2,Spicy Grilled Pineapple Salsa Recipe,https://www.archanaskitchen.com/spicy-grilled-...,good,2091,4.945959,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,10 M,0 M,Extra Virgin Olive Oil|Pineapple|White onion|R...,To begin making the Spicy Grilled Pineapple Sa...,Archana's Kitchen,Party Starter & Appetizer Recipes|Pineapple Re...,Mexican Recipes
3,Karwar Style Dali Thoy Recipe - Toor dal Curry,https://www.archanaskitchen.com/dali-thoy-reci...,good,990,4.888889,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,5 M,20 M,Arhar dal (Split Toor Dal)|Turmeric powder (Ha...,To prepare Karwar Style Dali Thoy Recipe (Toor...,Jyothi Rajesh,Side Dish Recipes|South Indian Recipes|Indian ...,Indian Curry Recipes
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,https://www.archanaskitchen.com/rajma-kofta-in...,good,345,4.828986,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,20 M,30 M,Rajma (Large Kidney Beans)|Cashew nuts|Sultana...,To begin making Rajma Kofta In Milk And Poppy ...,RUBY PATHAK,Side Dish Recipes|Indian Lunch Recipes|Office ...,Kofta Recipes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8004,Zucchini and Carrot Pasta Recipe,https://www.archanaskitchen.com/zucchini-and-c...,good,738,4.934959,is a healthy recipe which can be served over ...,Italian Recipes,Dinner,Vegetarian,10 M,30 M,Spirali pasta|Onions|Tomatoes|Green zucchini|C...,To begin with Zucchini And Carrot Pasta Recipe...,Aarti Sharma,Vegetarian Recipes|Carrot Recipes|Zucchini Rec...,Pasta Recipes
8005,Hyderabadi Khatti Dal Recipe (Hyderabadi Tangy...,https://www.archanaskitchen.com/hyderabadi-kha...,good,5173,4.903151,Hyderabadi Khatti Dal is a delicious tangy dal...,Hyderabadi,Lunch,High Protein Vegetarian,20 M,40 M,Arhar dal (Split Toor Dal)|Tomato|Turmeric pow...,To begin making the Hyderabadi Khatti Dal Reci...,Farrukh Aziz Ansari,Gluten Free Recipes|Indian Diabetic Recipes,Indian Curry Recipes
8006,Karwar Style Muga Ambat Recipe (Moong Sprouts ...,https://www.archanaskitchen.com/karwar-style-m...,good,1202,4.946755,is a curry recipe made from 'Muga' - which re...,Karnataka,Side Dish,High Protein Vegetarian,10 M,15 M,Turmeric powder (Haldi)|Sugar|Salt|Fresh cocon...,To prepare Karwar Style Muga Ambat Recipe (Moo...,Archana's Kitchen,High Protein Vegetarian Diet Recipes|South Ind...,Indian Curry Recipes
8007,"Beetroot, Amla And Pudina Juice Recipe | Mint ...",https://www.archanaskitchen.com/beetroot-amla-...,good,1305,4.925670,is a healthy juice recipe which acts as a gre...,Indian,Snack,Vegetarian,10 M,0 M,Beetroot|Amla (Nellikai/ Gooseberry)|Mint Leav...,"To begin making the Beetroot, Amla And Pudina...",Archana's Kitchen,Healthy Recipes|Beetroot Recipes|Pudina Mint R...,Indian Drink Recipes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8009 entries, 0 to 8008
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recipe_title   8009 non-null   object 
 1   url            8009 non-null   object 
 2   record_health  8009 non-null   object 
 3   vote_count     8009 non-null   int64  
 4   rating         8009 non-null   float64
 5   description    7994 non-null   object 
 6   cuisine        7943 non-null   object 
 7   course         7854 non-null   object 
 8   diet           7858 non-null   object 
 9   prep_time      7979 non-null   object 
 10  cook_time      7979 non-null   object 
 11  ingredients    7997 non-null   object 
 12  instructions   8009 non-null   object 
 13  author         8009 non-null   object 
 14  tags           7930 non-null   object 
 15  category       8009 non-null   object 
dtypes: float64(1), int64(1), object(14)
memory usage: 1001.3+ KB


In [4]:
df.drop(columns= ['url', 'record_health', 'vote_count', 'rating', 'author', 'tags', 'prep_time'], inplace= True, errors='ignore')
df.head()

,recipe_title,description,cuisine,course,diet,cook_time,ingredients,instructions,category
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15 M,Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...,To begin making the Roasted Peppers And Mushro...,Pizza Recipes
1,Thakkali Gotsu Recipe | Thakkali Curry | Spicy...,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,20 M,Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...,To begin making Tomato Gotsu Recipe/ Thakkali ...,Indian Curry Recipes
2,Spicy Grilled Pineapple Salsa Recipe,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,0 M,Extra Virgin Olive Oil|Pineapple|White onion|R...,To begin making the Spicy Grilled Pineapple Sa...,Mexican Recipes
3,Karwar Style Dali Thoy Recipe - Toor dal Curry,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,20 M,Arhar dal (Split Toor Dal)|Turmeric powder (Ha...,To prepare Karwar Style Dali Thoy Recipe (Toor...,Indian Curry Recipes
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,30 M,Rajma (Large Kidney Beans)|Cashew nuts|Sultana...,To begin making Rajma Kofta In Milk And Poppy ...,Kofta Recipes


In [5]:
df.isnull().sum()

,0
recipe_title,0
description,15
cuisine,66
course,155
diet,151
cook_time,30
ingredients,12
instructions,0
category,0


In [6]:
df['cook_time'] = df['cook_time'].astype(str).str.replace(' M', '', regex=False).astype(float)
numerical_cols = df.select_dtypes(include=['number']).columns.tolist()
print(f"Numerical Columns: {numerical_cols}")

Numerical Columns: ['cook_time']


In [7]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical Columns: {categorical_cols}")

Categorical Columns: ['recipe_title', 'description', 'cuisine', 'course', 'diet', 'ingredients', 'instructions', 'category']


In [8]:
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)

for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

/tmp/ipython-input-1282145286.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-1282145286.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [9]:
df.isnull().sum()

,0
recipe_title,0
description,0
cuisine,0
course,0
diet,0
cook_time,0
ingredients,0
instructions,0
category,0


In [10]:
print(type(categorical_cols[0]))
print(type(numerical_cols[0]))

<class 'str'>
<class 'str'>


In [11]:
data = numerical_cols + categorical_cols
data = pd.DataFrame(data)
data

,0
0,cook_time
1,recipe_title
2,description
3,cuisine
4,course
5,diet
6,ingredients
7,instructions
8,category


In [12]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df['clean_recipe_title'] = df['recipe_title'].apply(preprocess_text)
df['clean_instructions'] = df['instructions'].apply(preprocess_text)
df.head()

,recipe_title,description,cuisine,course,diet,cook_time,ingredients,instructions,category,clean_recipe_title,clean_instructions
0,Roasted Peppers And Mushroom Tortilla Pizza Re...,is a quicker version pizza to satisfy your cr...,Mexican,Dinner,Vegetarian,15.0,Tortillas|Extra Virgin Olive Oil|Garlic|Mozzar...,To begin making the Roasted Peppers And Mushro...,Pizza Recipes,roasted peppers and mushroom tortilla pizza re...,to begin making the roasted peppers and mushro...
1,Thakkali Gotsu Recipe | Thakkali Curry | Spicy...,also known as the is a quick and easy to ma...,South Indian Recipes,Lunch,Vegetarian,20.0,Sesame (Gingelly) Oil|Mustard seeds (Rai/ Kadu...,To begin making Tomato Gotsu Recipe/ Thakkali ...,Indian Curry Recipes,thakkali gotsu recipe thakkali curry spicy ...,to begin making tomato gotsu recipe thakkali c...
2,Spicy Grilled Pineapple Salsa Recipe,Spicy Grilled Pineapple Salsa is a simple reci...,Mexican,Side Dish,Vegetarian,0.0,Extra Virgin Olive Oil|Pineapple|White onion|R...,To begin making the Spicy Grilled Pineapple Sa...,Mexican Recipes,spicy grilled pineapple salsa recipe,to begin making the spicy grilled pineapple sa...
3,Karwar Style Dali Thoy Recipe - Toor dal Curry,The is a quintessential of Konkani dish whic...,Coastal Karnataka,Side Dish,High Protein Vegetarian,20.0,Arhar dal (Split Toor Dal)|Turmeric powder (Ha...,To prepare Karwar Style Dali Thoy Recipe (Toor...,Indian Curry Recipes,karwar style dali thoy recipe toor dal curry,to prepare karwar style dali thoy recipe toor ...
4,Rajma Kofta In Milk And Poppy Seed Gravy Recipe,Koftas are traditional Indian recipes mostly w...,North Indian Recipes,Side Dish,High Protein Vegetarian,30.0,Rajma (Large Kidney Beans)|Cashew nuts|Sultana...,To begin making Rajma Kofta In Milk And Poppy ...,Kofta Recipes,rajma kofta in milk and poppy seed gravy recipe,to begin making rajma kofta in milk and poppy ...


In [13]:
train_data, test_data = train_test_split(df, test_size= 0.2, random_state= 42)
train_data.shape, test_data.shape

((6407, 11), (1602, 11))

In [16]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [17]:
train_data, test_data = train_test_split(df, test_size= 0.2, random_state= 42)

train_encoding = tokenizer(train_data['clean_recipe_title'].tolist(),
                           truncation= True,
                           padding= True,
                           max_length= 128)
train_labels = tokenizer(train_data['clean_instructions'].tolist(),
                         truncation= True,
                         padding= True,
                         max_length= 128)
test_encoding = tokenizer(test_data['clean_recipe_title'].tolist(),
                         truncation= True,
                         padding= True,
                         max_length= 128)

test_labels = tokenizer(test_data['clean_instructions'].tolist(),
                        truncation= True,
                         padding= True,
                         max_length= 128)

In [18]:
train_dataset = Dataset.from_dict({
    'input_ids': train_encoding['input_ids'],
    'attention_mask': train_encoding['attention_mask'],
    'labels': train_labels['input_ids']
})

test_dataset = Dataset.from_dict({
    'input_ids' : test_encoding['input_ids'],
    'attention_mask' : test_encoding['attention_mask'],
    'labels' : test_labels['input_ids']
})

train_dataset[0]

{'input_ids': [3,
  15414,
  1483,
  1748,
  23,
  3,
  157,
  32,
  172,
  107,
  1598,
  14748,
  23,
  4301,
  23,
  11849,
  6605,
  2696,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'labels': [12,
  1731,
  28,
  3,
  15414,
  1483,
  1748,
  23,
  3,
  157,
  32,
  172,
  107,
  1598,
  14748,
  23,
  20743,
  8,
  6605,
  16,
  8,
  16188,
  22956,
  12,
  143,
  6605,
  5902,
  9,
  20743,
  8,
  3,
  26,
  5405,
  396,
  12,
  143,
  3,
  26,
  138,
  5902,
  9,
  88,
  144,
  9417,
  1043,
  16,
  3,
  9,
  617,
  23756,
  7299,
  995,
  34,
  12,
  3,
  7,
  102,
  20224,
  728,
  34,
  3,
  7,
  102,
  20224,
  7,
  617,
  387,
  1591,
  11388,
  15832,
  48,
  639,
  12,
  3,
  9,
  13374,
  617,
  6605,
  3,
 

In [19]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Enable input grads for gradient checkpointing with PEFT models
model.enable_input_require_grads()

traning_args = TrainingArguments(
    output_dir= './results',
    eval_strategy= 'epoch',
    save_strategy= 'epoch',
    learning_rate= 5e-5,
    num_train_epochs= 3,
    per_device_train_batch_size= 4, # Further reduced batch size
    per_device_eval_batch_size= 4,  # Further reduced batch size
    weight_decay= 0.01,
    warmup_steps= 100,
    logging_dir= './logs',
    logging_steps= 10,
    load_best_model_at_end= True,
    fp16=True, # Enable mixed precision training
    gradient_checkpointing=True # Enable gradient checkpointing
)

trainer = Trainer(
    model= model,
    args= traning_args,
    train_dataset= train_dataset,
    eval_dataset= test_dataset,
    data_collator= data_collator
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arsalbaig5145 (arsalbaig5145-arsal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,0.001000,0.000025
2,0.000200,0.000011
3,0.000200,0.000008


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=9612, training_loss=0.00991796924408868, metrics={'train_runtime': 1310.7853, 'train_samples_per_second': 14.664, 'train_steps_per_second': 7.333, 'total_flos': 193073009983488.0, 'train_loss': 0.00991796924408868, 'epoch': 3.0})

In [20]:
model.save_pretrained('./finetuned_model')
tokenizer.save_pretrained('./finetuned_model')

('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/spiece.model',
 './finetuned_model/added_tokens.json')

In [40]:
import torch

def generate_recipe(prompt, model, tokenizer, max_output_length=512):
    input_ids = tokenizer(
        prompt,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    ).input_ids

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids.to(device)
    model.to(device)

    # 3. Generate the output with improved parameters
    generated_ids = model.generate(
        input_ids=input_ids,
        max_length=max_output_length,
        num_beams=4,
        early_stopping=True,
        # length_penalty=0.8,
        # no_repeat_ngram_size=2,
        # temperature=1.0,
        # repetition_penalty=1.5
    )

    # 4. Decode and clean the result
    generated_recipe = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True)

    return generated_recipe

In [41]:
random_prompt = df.sample(n= 5, random_state= 42)
random_prompt

,recipe_title,description,cuisine,course,diet,cook_time,ingredients,instructions,category,clean_recipe_title,clean_instructions
554,Baale Dindu Kadale Palya - Banana Stem Curry W...,Baale Dindu Kadale Palya is South Indian Style...,Coastal Karnataka,Lunch,Vegetarian,40.0,Banana Stem|Kala Chana (Brown Chickpeas)|Fresh...,To prepare Baale Dindu Kadale Palya (Banana St...,Palya Recipes,baale dindu kadale palya banana stem curry wi...,to prepare baale dindu kadale palya banana ste...
6780,Vegan Chocolate Fudgesicles Recipe,Vegan Chocolate Fudgesicles Recipe is what you...,Continental,Snack,Vegan,0.0,Kabuli Chana (White Chickpeas)|Peanut Butter|S...,To begin making the Vegan Chocolate Fudgesicle...,Dessert Recipes,vegan chocolate fudgesicles recipe,to begin making the vegan chocolate fudgesicle...
3101,इलानीर रसम रेसिपी - Elaneer Rasam Recipe,एक ताज़ा रसम रेसिपी है जिसमे नारियल के पानी का...,South Karnataka,Lunch,Vegetarian,30.0,टमाटर|नारियल का पानी|नारियल|इमली|कढ़ी पत्ता|हीं...,इलानीर रसम रेसिपी बनाने के लिए सबसे पहले नारिय...,Rasam Recipes,elaneer rasam recipe,2 ...
7048,Pineapple Paneer Tikka Recipe,is a super easy starter that you can make qui...,North Indian Recipes,Appetizer,High Protein Vegetarian,30.0,Pineapple|Red Bell pepper (Capsicum)|Sunflower...,To begin making the Pineapple Paneer Tikka Rec...,Indian Snack Recipes,pineapple paneer tikka recipe,to begin making the pineapple paneer tikka rec...
2605,Konaseema Kodi Kura Recipe - Andhra Style Chic...,Konaseema Kodi Kura is a quite a popular recip...,Andhra,Dinner,High Protein Non Vegetarian,30.0,Chicken|Onion|Green Chillies|Dry Red Chilli|Po...,To begin making the Konaseema Kodi Kura recipe...,Chicken Recipes,konaseema kodi kura recipe andhra style chick...,to begin making the konaseema kodi kura recipe...


In [42]:
prompt = "Vegan Chocolate Fudgesicles Recipe"
generated_recipe = generate_recipe(prompt, model, tokenizer)
print(f"Prompt: {prompt}")
print(f"Generated Recipe: {generated_recipe}")

Prompt: Vegan Chocolate Fudgesicles Recipe
Generated Recipe: Vegan Chocolate Fudgesicles Recipe
